# imports

In [1]:
%matplotlib qt5
from class_PDC import PDC
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from drawnow import drawnow
import importlib
import threading
import time
import socket
import datetime


# test pdc class

In [8]:
#test if host name and port is valid
# resp  = socket.getaddrinfo('mundomain.arlab1.xyz', 5715)
# print('Family: {}'.format(resp[0][0]))
# print('Canonname:{}'.format(resp[0][4]))


In [4]:
p = PDC("192.168.10.120",port=5714)
#p = PDC('mundomain.arlab1.xyz',port=5715)
p.connect()
p.start_reading()
#time.sleep(2)
#p.stop_reading()
#p.disconnect()

In [4]:
p.start_streaming_server()

starting up on localhost port 5000


In [6]:
p.disconnect()

In [11]:
p.disconnect()

# Stream splitter

In [ ]:
import socket
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the port
server_address = ('localhost', 5000)
print('starting up on {} port {}'.format(*server_address))
sock.bind(server_address)

# Listen for incoming connections
sock.listen(2)

while True:
    # Wait for a connection
    print('waiting for a connection')
    connection, client_address = sock.accept()
    try:
        print('connection from', client_address)

        # Receive the data in small chunks and retransmit it
        while True:
            data = connection.recv(16)
            print('received {!r}'.format(data))
            if data:
                data=p.cfg2_raw
                print('sending data back to the client')
                connection.sendall(data)
                
            else:
                print('no data from', client_address)
                break

    finally:
        # Clean up the connection
        #connection.close()#%%
        pass


In [ ]:
import socket
import threading

def start_streaming_server(port=5000):
    # Create a TCP/IP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # Bind the socket to the port
    server_address = ('localhost', port)
    print('starting up on {} port {}'.format(*server_address))
    sock.bind(server_address)

    # Listen for incoming connections
    sock.listen(2)
    lst_connections =[]
    lst_connections_threads =[]
    while True:
        # Wait for a connection
        print('waiting for a connection')
        #connection, client_address = sock.accept()
        lst_connections.append(sock.accept())
        client_address = lst_connections[-1][1]

        try:
            print('connection from', client_address)
            lst_connections[-1][0].sendall(p.cfg2_raw)
            time.sleep(1)

            lst_connections_threads.append(threading.Thread(target=stream_dataframe,args=(lst_connections[-1][0],p
                                                                                          .dataframe_raw)))
            print('Streaming')
            lst_connections_threads[-1].start()

        finally:
            # Clean up the connection
            #connection.close()#%%
            pass


In [ ]:
thd_streaming_server = threading.Thread(target=start_streaming_server())
thd_streaming_server.start()

# PDC alignment test

In [2]:
pdc_readings = PDC("192.168.10.120",port=5716)
pdc_readings.connect()
wtpark_fw_readings = PDC("192.168.10.120",port=7714,idcode=4)
wtpark_fw_readings.connect()
pdc_readings.start_reading()
wtpark_fw_readings.start_reading()

KeyboardInterrupt: 

In [3]:
wtpark_fw_readings.cfg2_raw

b'\xaa\x01\x00J\x00\x04d$\x82\x90\x08\x0b\x92=\x01\x80G\x83X1\xc0\x03guG\x83X1@\x08\xad\xb0G\x83X1=(\xc7<B\x8eQF\xbf\xb79IB\x8eQF@0x\x80B\x8eQF?)\xb7\xb6Bp=q\x00\x00\x00\x00\x99F'

In [8]:
wtpark_fw_readings.cfg2_raw[21:37].decode('windows-1252')

'\x03guGƒX1@\x08\xad°GƒX1='

In [3]:

# t1 = datetime.datetime.now()
# delta_t = 0
# secs_lapsed = 0
# while secs_lapsed<10:
#     t2 = datetime.datetime.now()
#     delta_t= t2-t1
#     secs_lapsed=delta_t.seconds
#time.sleep(20)



ok
ok
ok


In [13]:
pdc_readings.stop_reading()
wtpark_fw_readings.stop_reading()

In [7]:
pdcgen1= pdc_readings.dataframes_read[pdc_readings.dataframes_read.STN=='OPPMU_WTPARK    ']
wtparkdata = wtpark_fw_readings.dataframes_read

In [13]:
plt.figure()
plt.plot(pdcgen1.TIME,pdcgen1.PHASOR1_MAG)
#plt.plot(gen1data.TIME,gen1data.PHASOR1_MAG)
plt.show()

In [12]:
wtpark_fw_readings.dataframes_read

,TIME,FRACSEC,STN,IDCODE,DATA_VALID,PMU_ERROR,PMU_SYNC,DATA_SORTING,TRIGGER_DETECTED,CONFIG_CHANGED,...,PHASOR5_CHNAM,PHASOR5_UNIT,PHASOR5_MAG,PHASOR5_ANG,PHASOR6_CHNAM,PHASOR6_UNIT,PHASOR6_MAG,PHASOR6_ANG,FREQ,DFREQ
EPOCH_TIME,,,,,,,,,,,,,,,,,,,,,
1.680113e+09,2023-03-29 18:11:23,0.65,OPPMU_WTPARK,4,True,False,True,timestamp,False,False,...,IB_WTPARK,volt,71.158737,157.984789,IC_WTPARK,volt,71.158737,37.984779,60.060001,0.0


In [9]:
pdc_readings.dataframes_read

,TIME,FRACSEC,STN,IDCODE,DATA_VALID,PMU_ERROR,PMU_SYNC,DATA_SORTING,TRIGGER_DETECTED,CONFIG_CHANGED,...,PHASOR5_CHNAM,PHASOR5_UNIT,PHASOR5_MAG,PHASOR5_ANG,PHASOR6_CHNAM,PHASOR6_UNIT,PHASOR6_MAG,PHASOR6_ANG,FREQ,DFREQ
EPOCH_TIME,,,,,,,,,,,,,,,,,,,,,
1.680113e+09,2023-03-29 18:04:08,0.595833,OPPMU_WTPARK,4,True,False,False,timestamp,False,False,...,Phasor4,amp,71.158737,157.984789,Phasor5,amp,71.158737,37.984775,60.060001,0.0
1.680113e+09,2023-03-29 18:04:08,0.595833,OPPMU_GEN2,2,True,False,False,timestamp,False,False,...,Phasor4,amp,981.677185,131.656281,Phasor5,amp,981.677185,11.656267,60.060001,0.0
1.680113e+09,2023-03-29 18:04:08,0.595833,OPPMU_GEN1,1,True,False,False,timestamp,False,False,...,Phasor4,amp,1079.404663,136.030840,Phasor5,amp,1079.404663,16.030828,60.060001,0.0
1.680113e+09,2023-03-29 18:04:08,0.595833,OPPMU_SLACK,3,True,False,False,timestamp,False,False,...,Phasor4,amp,1103.424561,120.246560,Phasor5,amp,1103.424561,0.246569,60.060001,0.0


In [14]:
pdcgen1.PHASOR1_MAG

EPOCH_TIME
1.680113e+09    67248.382812
1.680113e+09    67248.382812
Name: PHASOR1_MAG, dtype: float64

In [5]:
pdc

In [6]:
c.seconds

10